# Interactias Geo Selected Network

One way of examining the impact of invasive species is to look at all their interactions and the interations those organisms have with each other. This full interaction network gives you an indication of whether a species might be a "keystone" species and therefore have a disproportionatly large impact.

In this step of the evolution of this script it will be adapted to quantify species by their occupancy.

I will harvest species interactions data from GloBI (https://www.globalbioticinteractions.org/) to discover the species that interact with an invasive species.
I will then harvest all the interactions for those species to create two tiers of interactions.
I will then count all the occurences of these in species in the Belgian datacube.
I will then visualize this.

This notebook takes considerable insperation and code from Yikang Li's project on GloBI (https://curiositydata.org/part1_globi_access/).


### For information: Invasive alien species of Union concern
|A|H|O|
|--|--|--|
|Acacia saligna|Herpestes javanicus|Pacifastacus leniusculus|
|Acridotheres tristis|Humulus scandens|Parthenium hysterophorus|
|Ailanthus altissima|Hydrocotyle ranunculoides|Pennisetum setaceum|
|Alopochen aegyptiaca|Impatiens glandulifera|Perccottus glenii|
|Alternanthera philoxeroides|Lagarosiphon major|Persicaria perfoliata|
|Andropogon virginicus|Lepomis gibbosus|Plotosus lineatus|
|Arthurdendyus triangulatus|Lespedeza cuneata|Procambarus clarkii|
|Asclepias syriaca|Lithobates catesbeianus|Procambarus fallax|
|Baccharis halimifolia|Ludwigia peploides|Procyon lotor|
|Cabomba caroliniana|Lygodium japonicum|Prosopis juliflora|
|Callosciurus erythraeus|Lysichiton americanus|Pseudorasbora parva|
|Cardiospermum grandiflorum|Microstegium vimineum|Pueraria montana|
|Cortaderia jubata|Muntiacus reevesi|Salvinia molesta|
|Corvus splendens|Myocastor coypus|Sciurus carolinensis|
|Ehrharta calycina|Myriophyllum aquaticum|Sciurus niger|
|Eichhornia crassipes|Myriophyllum heterophyllum|Tamias sibiricus|
|Elodea nuttallii|Nasua nasua|Threskiornis aethiopicus|
|Eriocheir sinensis|Nyctereutes procyonoides|Trachemys scripta|
|Gunnera tinctoria|Ondatra zibethicus|Triadica sebifera|
|Gymnocoronis spilanthoides|Orconectes limosus|Vespa velutina|
|Heracleum mantegazzianum|Orconectes virilis||
|Heracleum persicum|Oxyura jamaicensis||

In [652]:
import sys
print(sys.version)

#Python 3.7.4 (default, Aug 9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
#pygbif 0.3.0

3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]


In [653]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from pygbif import species
from pygbif import occurrences as occ
import sqlite3
from sqlite3 import Error
import pydot
import math

### Setting up some parameters

In [654]:
# There is no point accepting all the species that have more than one record.
# There are too many casual records of plants and birds
thresholdForOccNum = 5
# If you want to exclude an interaction add it here
#interactionsToExclude = "visitsFlowersOf" #If you're not intersted in pollinators this will remove most of them
interactionsToExclude = "interactsWith"    #Many interactions in GloBI do not specify a specific interaction. You can get a cleaner netwot by excluding them
#interactionsToExclude = ""

In [655]:

## Define the country of interest
country  = 'Belgium'

## The occurrence cube has data for each year, but here we aggreagate years to get a more complete measure of occupancy.
## This parameter defines the year from where to consider records for the occupancy
year = 2000

## Define the place to find the data cube for occupancy data
## Currently the cube contains only Belgian data so that is all that can be used#
# A script to create cube.db is here https://github.com/AgentschapPlantentuinMeise/occcube
database = r"..\..\data\cube.db" 

## A script to create globi.db is here https://github.com/AgentschapPlantentuinMeise/createGlobiDB
## Define the place to find the interaction data
globiDB = r"..\data\globi.db"

### Define the taxon for the notebook for which to find all interactions


In [656]:
taxon=input()

Sceliphron caementarium


In [657]:
taxon = taxon.strip()

## Check to see if the taxon exits in GBIF

In [658]:
try:
    #NOTE: The function method name_suggest has a different behavior with regards to the gender of Latin names to name_backbone
    # If name_backbone is given a name in one gender it can still return the best match from the GBIF backbone if that
    # name is spelled as if it was another gender.
    #key = species.name_suggest(q=taxon, limit = 1)
    match = species.name_backbone(name=taxon, limit = 1)
    #print(key)
    
    # if there is no match this is returned form .name_backbone {'confidence': 100, 'matchType': 'NONE', 'synonym': False}
    if match['matchType'] == 'NONE':
        raise ValueError("TAXON NOT FOUND ON GBIF!")
    else:
        key = match['usageKey']
except ValueError as ve:
    print(ve)
    exit(1)




In [659]:
fullScientificName = match['scientificName']
print('The taxon to be studied is ' + fullScientificName)


The taxon to be studied is Sceliphron caementarium (Drury, 1773)


### Just for information take a look at the interactions that the taxon has

In [660]:
# What are all the types of interactions involving taxon as source taxon?
#data[data['sourceTaxonName'] == taxon]['interactionTypeName'].unique()
try:
    connGlobi = sqlite3.connect(globiDB)
except Error as e:
    print(e)

In [661]:
curGlobi = connGlobi.cursor()
curGlobi.execute("SELECT interactionTypeName, COUNT(interactionTypeName) from globi  WHERE sourceTaxonSpeciesName = ? GROUP BY interactionTypeName;", (taxon,))
sourceInteractions = curGlobi.fetchall()

In [662]:
sourceInteractions

[('createsHabitatFor', 15),
 ('eats', 82),
 ('hasHost', 2),
 ('interactsWith', 135),
 ('preysOn', 244),
 ('visits', 114),
 ('visitsFlowersOf', 184)]

In [663]:
curGlobi = connGlobi.cursor()
curGlobi.execute("SELECT interactionTypeName, COUNT(interactionTypeName) from globi  WHERE targetTaxonSpeciesName = ? GROUP BY interactionTypeName;", (taxon,))
targetInteractions = curGlobi.fetchall()
targetInteractions

[('interactsWith', 4),
 ('kleptoparasiteOf', 2),
 ('parasiteOf', 12),
 ('parasitoidOf', 21),
 ('pathogenOf', 2),
 ('preysOn', 11),
 ('visitsFlowersOf', 2)]

## Get the primary interation data for the species in question

This is limited to the rank of species because otherwise the network can get very large with rather irrelivent higher taxa.
However, if no interactions are found at the species level it might be worth removing this restriction.

In [664]:
curGlobi = connGlobi.cursor()
#curGlobi.execute("SELECT * from globi  WHERE targetTaxonName = ? and interactionTypeName NOT IN (?);", \
if interactionsToExclude == "":
    curGlobi.execute("SELECT * from globi  WHERE targetTaxonSpeciesName = ?;", \
                 (taxon,))
else:
    curGlobi.execute("SELECT * from globi  WHERE targetTaxonSpeciesName = ? and interactionTypeName NOT IN (?);", \
                 (taxon,interactionsToExclude,))
interactDataTaxon = curGlobi.fetchall()

In [665]:
curGlobi = connGlobi.cursor()
#curGlobi.execute("SELECT * from globi  WHERE sourceTaxonName = ? and interactionTypeName NOT IN (?);", \
if interactionsToExclude == "":
    curGlobi.execute("SELECT * from globi  WHERE sourceTaxonSpeciesName = ?;", \
                 (taxon,))
else:
    curGlobi.execute("SELECT * from globi  WHERE sourceTaxonSpeciesName = ? and interactionTypeName NOT IN (?);", \
                 (taxon,interactionsToExclude,))
sources = curGlobi.fetchall()

In [666]:
interactDataTaxon.extend(sources)
len(interactDataTaxon)

691

In [667]:
# Convert to a Pandas dataframe
interactDataTaxon = pd.DataFrame(interactDataTaxon)

In [668]:
# Add column names
interactDataTaxon
interactDataTaxon.columns = ['sourceTaxonId', \
                                'sourceTaxonIds','sourceTaxonName','sourceTaxonRank','sourceTaxonPathNames', \
                                'sourceTaxonPathIds','sourceTaxonPathRankNames','sourceTaxonSpeciesName','sourceTaxonSpeciesId',\
                                'sourceTaxonGenusName','sourceTaxonGenusId','sourceTaxonFamilyName','sourceTaxonFamilyId',\
                                'sourceTaxonOrderName','sourceTaxonOrderId','sourceTaxonClassName','sourceTaxonClassId',\
                                'sourceTaxonPhylumName','sourceTaxonPhylumId','sourceTaxonKingdomName','sourceTaxonKingdomId',\
                                'sourceId','sourceOccurrenceId','sourceInstitutionCode','sourceCollectionCode',\
                                'sourceCatalogNumber','sourceBasisOfRecordId',\
                                'sourceBasisOfRecordName','sourceLifeStageId','sourceLifeStageName','sourceBodyPartId',\
                                'sourceBodyPartName','sourcePhysiologicalStateId','sourcePhysiologicalStateName', \
                                'sourceSexId', 'sourceSexName','interactionTypeName',\
                                'interactionTypeId','targetTaxonId','targetTaxonIds','targetTaxonName',\
                                'targetTaxonRank','targetTaxonPathNames','targetTaxonPathIds','targetTaxonPathRankNames',\
                                'targetTaxonSpeciesName','targetTaxonSpeciesId','targetTaxonGenusName','targetTaxonGenusId',\
                                'targetTaxonFamilyName','targetTaxonFamilyId','targetTaxonOrderName','targetTaxonOrderId',\
                                'targetTaxonClassName','targetTaxonClassId','targetTaxonPhylumName','targetTaxonPhylumId',\
                                'targetTaxonKingdomName','targetTaxonKingdomId','targetId','targetOccurrenceId',\
                                'targetInstitutionCode','targetCollectionCode',\
                                'targetCatalogNumber','targetBasisOfRecordId','targetBasisOfRecordName','targetLifeStageId',\
                                'targetLifeStageName','targetBodyPartId','targetBodyPartName','targetPhysiologicalStateId',\
                                'targetPhysiologicalStateName', 'targetSexId', 'targetSexName',\
                                'decimalLatitude','decimalLongitude','localityId',\
                                'localityName','eventDateUnixEpoch','argumentTypeId','referenceCitation',\
                                'referenceDoi','referenceUrl','sourceCitation','sourceNamespace',\
                                'sourceArchiveURI','sourceDOI','sourceLastSeenAtUnixEpoch']

## Get a list of all the primary interacting species

In [669]:
interactingTaxaData = interactDataTaxon.drop_duplicates()

In [670]:
primaryInteractingTaxa = interactingTaxaData['sourceTaxonName'].drop_duplicates()
len(primaryInteractingTaxa)

37

In [671]:
primaryInteractingTaxa = (primaryInteractingTaxa.append(interactingTaxaData['targetTaxonName']).drop_duplicates())
len(primaryInteractingTaxa)

224

In [672]:
primaryInteractingTaxa

0      Heterotheca sessiliflora
1           Melittobia chalybii
2         Melittobia megachilis
3                    Melittobia
5               Dibrachys pelos
                 ...           
677            Torilis arvensis
682         Verbesina virginica
684           Xysticus bharatae
688              Xysticus ferox
690           Zosis geniculatus
Length: 224, dtype: object

## A function to query the globi database

In [673]:
def secondaryDataInGlobi(conn, intaxon):
    cur = conn.cursor()
    if interactionsToExclude == "":
        cur.execute("SELECT * from globi  WHERE (targetTaxonSpeciesName = ? OR sourceTaxonSpeciesName = ?);", \
                 (intaxon,intaxon,))
    else:
        cur.execute("SELECT * from globi where (targetTaxonSpeciesName = ? OR sourceTaxonSpeciesName = ?) and interactionTypeName NOT IN (?);", \
                (intaxon,intaxon,interactionsToExclude,))
    return(cur.fetchall())

## Get all the secondary interactions

In [674]:
try:
    conn = sqlite3.connect(globiDB)
except Error as e:
    print(e)

In [675]:
secondarylist = []

for name in primaryInteractingTaxa:
    temp = secondaryDataInGlobi(conn, name)
    secondarylist.extend(temp)

In [676]:
# Convert to a Pandas dataframe
secondarylist = pd.DataFrame(secondarylist)

In [677]:
secondarylist.columns = ['sourceTaxonId', \
                                'sourceTaxonIds','sourceTaxonName','sourceTaxonRank','sourceTaxonPathNames', \
                                'sourceTaxonPathIds','sourceTaxonPathRankNames','sourceTaxonSpeciesName','sourceTaxonSpeciesId',\
                                'sourceTaxonGenusName','sourceTaxonGenusId','sourceTaxonFamilyName','sourceTaxonFamilyId',\
                                'sourceTaxonOrderName','sourceTaxonOrderId','sourceTaxonClassName','sourceTaxonClassId',\
                                'sourceTaxonPhylumName','sourceTaxonPhylumId','sourceTaxonKingdomName','sourceTaxonKingdomId',\
                                'sourceId','sourceOccurrenceId','sourceInstitutionCode','sourceCollectionCode',\
                                'sourceCatalogNumber','sourceBasisOfRecordId',\
                                'sourceBasisOfRecordName','sourceLifeStageId','sourceLifeStageName','sourceBodyPartId',\
                                'sourceBodyPartName','sourcePhysiologicalStateId','sourcePhysiologicalStateName',\
                                'sourceSexId', 'sourceSexName', 'interactionTypeName',\
                                'interactionTypeId','targetTaxonId','targetTaxonIds','targetTaxonName',\
                                'targetTaxonRank','targetTaxonPathNames','targetTaxonPathIds','targetTaxonPathRankNames',\
                                'targetTaxonSpeciesName','targetTaxonSpeciesId','targetTaxonGenusName','targetTaxonGenusId',\
                                'targetTaxonFamilyName','targetTaxonFamilyId','targetTaxonOrderName','targetTaxonOrderId',\
                                'targetTaxonClassName','targetTaxonClassId','targetTaxonPhylumName','targetTaxonPhylumId',\
                                'targetTaxonKingdomName','targetTaxonKingdomId','targetId','targetOccurrenceId',\
                                'targetInstitutionCode','targetCollectionCode',\
                                'targetCatalogNumber','targetBasisOfRecordId','targetBasisOfRecordName','targetLifeStageId',\
                                'targetLifeStageName','targetBodyPartId','targetBodyPartName','targetPhysiologicalStateId',\
                                'targetPhysiologicalStateName', 'targetSexId', 'targetSexName', \
                                'decimalLatitude','decimalLongitude','localityId',\
                                'localityName','eventDateUnixEpoch','argumentTypeId','referenceCitation',\
                                'referenceDoi','referenceUrl','sourceCitation','sourceNamespace',\
                                'sourceArchiveURI','sourceDOI','sourceLastSeenAtUnixEpoch']

In [678]:
allInteractionsData = interactingTaxaData.append(secondarylist)

In [679]:
allInteractionsData = allInteractionsData.drop_duplicates()


In [680]:
allInteractionsData

,sourceTaxonId,sourceTaxonIds,sourceTaxonName,sourceTaxonRank,sourceTaxonPathNames,sourceTaxonPathIds,sourceTaxonPathRankNames,sourceTaxonSpeciesName,sourceTaxonSpeciesId,sourceTaxonGenusName,...,eventDateUnixEpoch,argumentTypeId,referenceCitation,referenceDoi,referenceUrl,sourceCitation,sourceNamespace,sourceArchiveURI,sourceDOI,sourceLastSeenAtUnixEpoch
0,INAT_TAXON:57988,GBIF:5401998 | INAT_TAXON:57988 | IRMNG:102088...,Heterotheca sessiliflora,species,Plantae | Tracheophyta | Magnoliopsida | Aster...,None,kingdom | phylum | class | order | family | ge...,Heterotheca sessiliflora,None,Heterotheca,...,1.597432e+12,https://en.wiktionary.org/wiki/support,edward_rooks. 2020. Sceliphron caementarium po...,None,None,http://iNaturalist.org is a place where you ca...,globalbioticinteractions/inaturalist,https://github.com/globalbioticinteractions/in...,None,2021-11-13T00:13:49.554Z
1,EOL:1072631,EOL:1072631 | GBIF:1383205 | WD:Q13833911 | do...,Melittobia chalybii,species,Animalia | Arthropoda | Insecta | Hymenoptera ...,EOL:1 | EOL:164 | EOL:344 | EOL:648 | EOL:692 ...,kingdom | phylum | class | order | superfamily...,Melittobia chalybii,EOL:1072631,Melittobia,...,NaN,https://en.wiktionary.org/wiki/support,"F. Seow-Choen, A Taxonomic Guide to the Stick...",None,None,Ed Baker; Ian J. Kitching; George W. Beccaloni...,globalbioticinteractions/natural-history-museu...,https://github.com/globalbioticinteractions/na...,None,2021-11-13T01:24:34.089Z
2,EOL:1072623,EOL:1072623 | GBIF:1383207 | NCBI:1031399 | WD...,Melittobia megachilis,species,Animalia | Arthropoda | Insecta | Hymenoptera ...,EOL:1 | EOL:164 | EOL:344 | EOL:648 | EOL:692 ...,kingdom | phylum | class | order | superfamily...,Melittobia megachilis,EOL:1072623,Melittobia,...,NaN,https://en.wiktionary.org/wiki/support,"F. Seow-Choen, A Taxonomic Guide to the Stick...",None,None,Ed Baker; Ian J. Kitching; George W. Beccaloni...,globalbioticinteractions/natural-history-museu...,https://github.com/globalbioticinteractions/na...,None,2021-11-13T01:24:34.089Z
3,GBIF:1383189,GBIF:1383189 | IRMNG:1108047 | NCBI:29042,Melittobia,genus,Animalia | Arthropoda | Insecta | Hymenoptera ...,GBIF:1 | GBIF:54 | GBIF:216 | GBIF:1457 | GBIF...,kingdom | phylum | class | order | family | genus,None,None,Melittobia,...,NaN,https://en.wiktionary.org/wiki/support,"F. Seow-Choen, A Taxonomic Guide to the Stick...",None,None,Ed Baker; Ian J. Kitching; George W. Beccaloni...,globalbioticinteractions/natural-history-museu...,https://github.com/globalbioticinteractions/na...,None,2021-11-13T01:24:34.089Z
5,EOL:845953,EOL:845953 | GBIF:1392829 | IRMNG:10086274 | N...,Dibrachys pelos,species,Animalia | Arthropoda | Insecta | Hymenoptera ...,EOL:1 | EOL:164 | EOL:344 | EOL:648 | EOL:692 ...,kingdom | phylum | class | order | superfamily...,Dibrachys pelos,EOL:845953,Dibrachys,...,NaN,https://en.wiktionary.org/wiki/support,"F. Seow-Choen, A Taxonomic Guide to the Stick...",None,None,Ed Baker; Ian J. Kitching; George W. Beccaloni...,globalbioticinteractions/natural-history-museu...,https://github.com/globalbioticinteractions/na...,None,2021-11-13T01:24:34.089Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54843,INAT_TAXON:129415,GBIF:1942532 | INAT_TAXON:129415 | IRMNG:10617...,Copaeodes aurantiaca,species,Animalia | Arthropoda | Insecta | Lepidoptera ...,None,kingdom | phylum | class | order | family | ge...,Copaeodes aurantiaca,None,Copaeodes,...,1.384128e+12,https://en.wiktionary.org/wiki/support,Greg Lasley. 2013. Copaeodes aurantiaca nectar...,None,None,http://iNaturalist.org is a place where you ca...,xrisfg/inaturalist,https://github.com/xrisfg/inaturalist/archive/...,None,2021-11-13T04:49:30.508Z
54850,NCBI:336613,GBIF:2165238 | IRMNG:10498785 | ITIS:892139 | ...,Xysticus ferox,species,root | cellular organisms | Eukaryota | Opisth...,NCBI:1 | NCBI:131567 | NCBI:2759 | NCBI:33154 ...,null | null | superkingdom | null | kingdom | ...,Xysticus ferox,NCBI:336613,

In [681]:
    # cleanInteractDataTaxon = allInteractionsData[['sourceTaxonId', 'sourceTaxonName', 'sourceTaxonSpeciesName', 'sourceTaxonRank',
    #    'sourceTaxonFamilyName', 'interactionTypeName',
    #    'targetTaxonName','targetTaxonSpeciesName','targetTaxonRank', 'sourceCitation', 'sourceDOI',
    #    'sourceTaxonKingdomName','sourceTaxonKingdomId', 'targetTaxonKingdomName','targetTaxonKingdomId'                                        
    #    ]].dropna(subset=['targetTaxonSpeciesName','sourceTaxonSpeciesName'])
    
    cleanInteractDataTaxon = allInteractionsData[['sourceTaxonName', 'sourceTaxonSpeciesName', 'sourceTaxonRank',
            'sourceTaxonFamilyName', 'interactionTypeName',
            'targetTaxonName','targetTaxonSpeciesName','targetTaxonRank', 'sourceCitation', 'sourceDOI',
            'sourceTaxonKingdomName', 'targetTaxonKingdomName']].dropna(subset=['targetTaxonSpeciesName','sourceTaxonSpeciesName'])
    
    

In [682]:
allSourceInteractingTaxa = cleanInteractDataTaxon['sourceTaxonSpeciesName'].drop_duplicates()
allTargetInteractingTaxa = cleanInteractDataTaxon['targetTaxonSpeciesName'].drop_duplicates()

In [683]:
cleanInteractDataTaxon = cleanInteractDataTaxon.drop_duplicates()
cleanInteractDataTaxon.count()

sourceTaxonName           11502
sourceTaxonSpeciesName    11502
sourceTaxonRank           11437
sourceTaxonFamilyName     11319
interactionTypeName       11502
targetTaxonName           11502
targetTaxonSpeciesName    11502
targetTaxonRank           11398
sourceCitation            11502
sourceDOI                  1717
sourceTaxonKingdomName    11278
targetTaxonKingdomName    11399
dtype: int64

In [684]:
allInteractingTaxa = allSourceInteractingTaxa.append(allTargetInteractingTaxa).drop_duplicates()
allInteractingTaxa.count()

5806

In [685]:
allInteractingTaxa.head()

0     Heterotheca sessiliflora
1          Melittobia chalybii
2        Melittobia megachilis
5              Dibrachys pelos
6    Monodontomerus montivagus
dtype: object

In [686]:
allInteractingTaxa

0         Heterotheca sessiliflora
1              Melittobia chalybii
2            Melittobia megachilis
5                  Dibrachys pelos
6        Monodontomerus montivagus
                   ...            
54369        Petrophora chlorosata
54375               Pipiza festiva
54377             Pyronia bathseba
54381           Zizeeria karsandra
54851         Artemisia tridentata
Length: 5806, dtype: object

In [687]:
# How many different sort of interaction do I have left?
# Checking out all the interaction types
cleanInteractDataTaxon.groupby(cleanInteractDataTaxon['interactionTypeName']).size().sort_values(ascending = False)

interactionTypeName
visitsFlowersOf       2909
eats                  2778
hasHost               1698
preysOn               1095
pathogenOf             869
visits                 810
parasiteOf             478
pollinates             365
mutualistOf            207
adjacentTo             174
parasitoidOf            39
coOccursWith            23
createsHabitatFor       20
hasDispersalVector      19
livesOn                  7
livesInsideOf            3
laysEggsOn               2
kleptoparasiteOf         2
livesUnder               1
hasVector                1
endoparasitoidOf         1
livesNear                1
dtype: int64

### This function takes a name string and checks on GBIF to see if the name exists there.

In [688]:
def speciesExistsInGBIF(name, rank):
    try:
        match = species.name_backbone(name=name, rank=rank, limit = 1)

        # if there is no match this is returned from .name_backbone {'confidence': 100, 'matchType': 'NONE', 'synonym': False}
        if match['matchType'] == 'NONE':
            return False
        else:
            return match
    except ValueError as ve:
        print(ve)
        exit(1)

### Check that the species in question is actually found on GBIF

In [689]:
if(speciesExistsInGBIF(taxon, "species") == False):
    print("##### {0} has not been found on GBIF #####".format(taxon))

### Citation for the GBIF Backbone Taxonomy

In [690]:
from datetime import date
print("GBIF Secretariat (2019). GBIF Backbone Taxonomy. Checklist dataset https://doi.org/10.15468/39omei accessed via GBIF.org on ", date.today())

GBIF Secretariat (2019). GBIF Backbone Taxonomy. Checklist dataset https://doi.org/10.15468/39omei accessed via GBIF.org on  2022-01-01


### Check to see which taxa in the interaction network are found in GBIF and list those ones that are not

In [691]:
taxaFound = {}
kingdom = {}

print('Taxa from GLoBI, but not found in GBIF')
for name in allInteractingTaxa.items():
    GBIFName = speciesExistsInGBIF(name[1], "species")
    if GBIFName == False:
        taxaFound[name[1]] = False
        print(name[1])
    else:
        taxaFound[name[1]] = GBIFName['usageKey']
        kingdom[name[1]] = GBIFName['kingdom']

    

Taxa from GLoBI, but not found in GBIF
Stilbum cyanurum
Beauveria bassiana victorivirus NZL/1980
Beauveria bassiana RNA virus 1
Nephila clavipes virus 4
Nephila clavipes virus 3
Nephila clavipes virus 2
Nephila clavipes virus 6
Nephila clavipes virus 1
Golden silk orbweaver associated circular virus 1
Pseudopanugus n.sp. (aff. irregularis)
Lygaeus turcicus
N/A N/A
Iris yellow spot virus
Onion yellows phytoplasma
Groundnut bud necrosis virus
Dill cryptic virus 1
Lygaeus kalmii
Melanomma seminis
Broccoli necrotic yellows virus
Turnip crinkle virus satellite RNA
Turnip ringspot virus
Lucerne transient streak virus satellite RNA
Spilarctia obliqua
Bacillus pichinotyi
Gordonia alkanivorans
Hendersonia peckii
Sphaerotheca
Ophiobolus
Carrot red leaf luteovirus associated RNA
Carrot mottle virus satellite RNA
Nectria coccinea
Pseudopanugus sp. E3
Pseudopanugus sp. E4
Dufourea sp.
Pseudopanugus sp. E1
Brachycaudus cardui
Graphium choredon
undetermined Cottoidei Lake Baikal
Nectria
Passion fruit

In [692]:
# Convert to a Pandas dataframe
taxaFound = pd.DataFrame.from_dict(taxaFound, orient='index')

In [693]:
len(taxaFound)

5806

In [694]:
kingdom

{'Heterotheca sessiliflora': 'Plantae',
 'Melittobia chalybii': 'Animalia',
 'Melittobia megachilis': 'Animalia',
 'Dibrachys pelos': 'Animalia',
 'Monodontomerus montivagus': 'Animalia',
 'Melittobia hawaiiensis': 'Animalia',
 'Crematogaster lineolata': 'Animalia',
 'Dryobates pubescens': 'Animalia',
 'Beauveria bassiana': 'Fungi',
 'Metarhizium anisopliae': 'Fungi',
 'Amobia floridensis': 'Animalia',
 'Acroricnus tricolor': 'Animalia',
 'Chalybion californicum': 'Animalia',
 'Chrysis coerulans': 'Animalia',
 'Chrysis angolensis': 'Animalia',
 'Diogmites angustipennis': 'Animalia',
 'Melittobia acasta': 'Animalia',
 'Latrodectus mactans': 'Animalia',
 'Kukulcania hibernalis': 'Animalia',
 'Trichonephila clavipes': 'Animalia',
 'Argiope aurantia': 'Animalia',
 'Tidarren sisyphoides': 'Animalia',
 'Diogmites basalis': 'Animalia',
 'Proctacanthus philadelphicus': 'Animalia',
 'Acroricnus seductor': 'Animalia',
 'Chrysis mediadentata': 'Animalia',
 'Melittobia australica': 'Animalia',
 'A

### This function takes a GBIF species key and counts how many occurences exist in the data cube

In [695]:
def speciesCountInCube(conn, key):
    cur = conn.cursor()
    cur.execute("SELECT COUNT(taxonKey) from (SELECT taxonKey FROM cube WHERE year > ? and taxonKey = ? GROUP BY eea_cell_code)", (year, key,))
    return(cur.fetchall())

### Loop over all the taxa that are in the interaction network and are in GBIF to find the ones that have been found in the country

In [696]:
taxaFoundInCountry = []

try:
    conn = sqlite3.connect(database)
except Error as e:
    print(e)

In [697]:
year

2000

In [698]:
for GBIFtaxon in taxaFound.iterrows():
    GBIFOccCount = speciesCountInCube(conn, GBIFtaxon[1][0])
    print(GBIFtaxon[0])
    if GBIFOccCount[0][0] > thresholdForOccNum and GBIFtaxon[0] != taxon: # The threshold for observations is not greater than one, due to all the casual records
        taxaFoundInCountry.append({'key': GBIFtaxon[1][0], 'species': GBIFtaxon[0], 'count': GBIFOccCount[0][0]})
        print('{0} with key {1} occurs in {2} km squares.'.format(GBIFtaxon[0],GBIFtaxon[1][0],GBIFOccCount[0][0]))
    elif GBIFtaxon[0] == taxon:
        taxaFoundInCountry.append({'key': GBIFtaxon[1][0], 'species': GBIFtaxon[0], 'count': GBIFOccCount[0][0]})
        print('{0} with key {1} occurs in {2} km squares.'.format(GBIFtaxon[0],GBIFtaxon[1][0],GBIFOccCount[0][0]))

Heterotheca sessiliflora
Melittobia chalybii
Melittobia megachilis
Dibrachys pelos
Monodontomerus montivagus
Melittobia hawaiiensis
Crematogaster lineolata
Dryobates pubescens
Beauveria bassiana
Metarhizium anisopliae
Amobia floridensis
Acroricnus tricolor
Stilbum cyanurum
Chalybion californicum
Chrysis coerulans
Chrysis angolensis
Diogmites angustipennis
Melittobia acasta
Latrodectus mactans
Kukulcania hibernalis
Trichonephila clavipes
Argiope aurantia
Tidarren sisyphoides
Diogmites basalis
Proctacanthus philadelphicus
Acroricnus seductor
Chrysis mediadentata
Melittobia australica
Amobia signata
Stenarella domator
Stenarella domator with key 1283774 occurs in 11 km squares.
Sceliphron caementarium
Sceliphron caementarium with key 5041466 occurs in 1 km squares.
Icaricia acmon
Megachile perihirta
Ochlodes sylvanoides
Brachycaudus helichrysi
Apodemia mormo
Puccinia grindeliae
Puccinia gutierreziae
Argiope argentata
Calliephialtes ferrugineus
Nephila clavipes
Agelaius phoeniceus
Argyrode

Achillea millefolium with key 3120060 occurs in 9248 km squares.
Agraulis vanillae
Grammoptera subargentata
Papilio zelicaon
Paranotoreas brephosata
Castilleja levisecta
Codophila varia
Junonia grisea
Stenopterus rufus
Stenopterus rufus with key 6097811 occurs in 26 km squares.
Strophiona tigrina
Andrena flavipes
Andrena flavipes with key 1357156 occurs in 3965 km squares.
Judolia instabilis
Icaricia saepiolus
Bombus californicus
Dolichovespula arenaria
Euderces picipes
Lycaena dispar
Vanessa virginiensis
Toxomerus occidentalis
Idaea ochrata
Helophilus latifrons
Fazia micrura
Adela flammeusella
Scaeva affinis
Spilomyia interrupta
Burnsius communis
Eupithecia centaureata
Polygonia faunus
Graphiphora augur
Ligyrocoris sylvestris
Stelis laticincta
Copestylum violaceum
Isodontia elegans
Anthophora bomboides
Cerceris nigrescens
Xestia smithii
Bombus nevadensis
Plebejus idas
Euodynerus hidalgo
Philanthus bilunatus
Harmonia axyridis
Harmonia axyridis with key 4989904 occurs in 7488 km squares

Emberiza schoeniclus
Emberiza schoeniclus with key 2491557 occurs in 11548 km squares.
Araneus diadematus
Araneus diadematus with key 2160133 occurs in 216 km squares.
Vespa velutina
Vespa velutina with key 1311477 occurs in 376 km squares.
Sceliphron spirifex
Pheidole pallidula
Dolomedes fimbriatus
Dolomedes fimbriatus with key 2150955 occurs in 67 km squares.
Episyron rufipes
Episyron rufipes with key 4505244 occurs in 126 km squares.
Episyron albonotatum
Argiope bruennichi
Argiope bruennichi with key 5170994 occurs in 299 km squares.
Xysticus ulmi
Xysticus ulmi with key 2165228 occurs in 149 km squares.
Pterostichus melanarius
Pterostichus melanarius with key 7856746 occurs in 13 km squares.
Argiope lobata
Sceliphron asiaticum
Argiope trifasciata
Bombus fraternus
Puccinia chloridis
Asclepias hirtella
Colias philodice
Sphex pennsylvanicus
Pompeius verna
Ancyloxypha numitor
Speyeria cybele
Satyrium calanus
Poanes viator
Euphyes conspicua
Euphyes dion
Cupido comyntas
Danaus gilippus
Ly

Leuroperna sera
Urocystis coralloides
Puccinia subnitens
Neopseudocercosporella capsellae
Peronospora
Periola tomentosa
Melanospora zamiae
Hyaloperonospora brassicae
Curvularia pallescens
Colletotrichum higginsianum
Cercosporella
Cercospora cruciferarum
Cercospora brassicicola
Cercospora brassicae-campestris
Cercospora brassicae
Arthrinium arundinis
Bagrada hilaris
Listroderes difficilis
Otiorhynchus cribricollis
Anthocharis scolymus
Nysius vinitor
Dictyotus caenosus
Delia sequoiae
Turnip crinkle virus satellite RNA
Turnip ringspot virus
Lucerne transient streak virus satellite RNA
Turnip yellow mosaic virus
Turnip yellows virus
Turnip leaf roll virus
Turnip rosette virus
Turnip curly top virus
Turnip vein-clearing virus
Turnip mosaic virus
Potato virus X
Brassica campestris chrysovirus
Xylocopa darwini
Entomoscelis americana
Atomaria nigrirostris
Aclypea opaca
Atomaria capitata
Atomaria coloradensis
Atomaria vespertina
Atomaria planulata
Francolinus francolinus
Callipepla californica


Megachile ligniseca
Megachile ligniseca with key 1335802 occurs in 273 km squares.
Anthidium manicatum
Anthidium manicatum with key 1334821 occurs in 2031 km squares.
Andrena marginata
Andrena hattorfiana
Andrena hattorfiana with key 1357364 occurs in 346 km squares.
Andrena dorsata
Andrena dorsata with key 1357872 occurs in 1239 km squares.
Galium spurium
Galium spurium with key 2913214 occurs in 40 km squares.
Andrena virginiana
Osmia georgica
Lasioglossum cinctipes
Orobanche reticulata
Sphaeronaema floccosum
Phyllosticta cirsii
Phomopsis cirsii
Ophiobolus cirsii
Diplodina cirsii
Erysiphe mayorii
Bremia lactucae
Urophora stylata
Xyphosia miliaria
Terellia tussilaginis
Tephritis cometa
Terellia ruficauda
Ensina sonchi
Chaetostomella cylindrica
Acanthiophilus helianthi
Stachybotrys dichroa
Puccinia punctiformis
Pyrenopeziza carduorum
Pyrenopeziza adenostylidis
Pirottaea brevipila
Pseudospiropes rousselianus
Phoma rubella
Pezizella discreta
Phoma cirsii
Pezizella glareosa
Mollisia clava

...

Phoenicurus phoenicurus
Phoenicurus phoenicurus with key 5739317 occurs in 6385 km squares.
Hylia prasina
Cossypha niveicapilla
Platysteira cyanea
Terpsiphone viridis
Prodotiscus insignis
Spermophaga ruficapilla
Dicrurus adsimilis
Cyanomitra olivacea
Chalcomitra senegalensis
Ploceus ocularis
Eurillas ansorgei
Estrilda nonnula
Turdoides sharpei
Malimbus rubricollis
Nectarinia amethystina
Lybius bidentatus
Campephaga flava
Nigrita fusconota
Eurillas virens
Turdus subalaris
Musophaga rossae
Neopelma aurifrons
Odontophorus capueira
Crithagra mozambica
Crithagra mozambica with key 6100996 occurs in 16 km squares.
Iduna pallida
Myiopagis caniceps
Eurillas latirostris
Sylvia atricapilla
Sylvia atricapilla with key 2492956 occurs in 14250 km squares.
Cinnyricinclus leucogaster
Pionus maximiliani
Phylloscopus sibillatrix
Phylloscopus sibillatrix with key 8128385 occurs in 1919 km squares.
Trachylaemus purpuratus
Cercopithecus ascanius
Colobus guereza
Turtur tympanistria
Papio anubis
Bycanistes 

Dipogon sayi
European bat 1 lyssavirus
Hypoponera opacior
Medicago sativa
Medicago sativa with key 9151957 occurs in 2463 km squares.
Megachile centuncularis
Megachile centuncularis with key 1335505 occurs in 1180 km squares.
Monomorium floricola
Nesepyris floridanus
Paratrechina bourbonica
Periplaneta americana
Pison koreense
Pseudomyrmex elongatus
Saperda candida
Sceliphron assimile
Sphecophaga vesparum
Trypoxylon collinum
Trypoxylon politum
Xenomyrmex floridanus
Zeta confusum
Anthophora abrupta
Osmia nigrifrons
Anthophora linsleyi
Anthophora occidentalis
Archips fumiferana
Megachile peruviana
Microdynerus bakerianus
Oryttus mirandus
Osmia kincaidii
Osmia ribifloris
Osmia sanrafaelae
Stelis depressa
Stenodynerus mystecus
Artona catoxantha
Bessa remota
Cocos nucifera
Erechthias flavistriata
Eumenes germaini
Megachile aetheria
Megachile gomphrenae
Pison argentatum
Pison spinolae
Ptychomyia remota
Cupressus sempervirens
Cydia pomonella
Cydia pomonella with key 1737847 occurs in 8 km squ

Anser anser
Anser anser with key 2498036 occurs in 10795 km squares.
Ceriana tridens
Lasioglossum incompletum
Protaphis debilicornis
Ballota acetabulosa
Delphinium peregrinum
Ecballium elaterium
Echium angustifolium
Heliotropium europaeum
Heliotropium europaeum with key 2925767 occurs in 6 km squares.
Hypericum triquetrifolium
Sideritis romana
Thymus capitatus
Boltonia asteroides
Eryngium pendletonensis
Euthamia graminifolia
Parthenium integrifolium
Pycnanthemum muticum
Scrophularia atrata
Scrophularia laevis
Scrophularia villosa
Solidago altissima
Solidago rugosa
Solidago sempervirens
Aegonychon purpurocaeruleum
Aubrieta deltoidea
Aubrieta deltoidea with key 3054117 occurs in 21 km squares.
Borago officinalis
Borago officinalis with key 2926110 occurs in 712 km squares.
Corydalis solida
Corydalis solida with key 5334208 occurs in 452 km squares.
Hyacinthus orientalis
Hyacinthus orientalis with key 2772666 occurs in 42 km squares.
Lamium galeobdolon
Lamium galeobdolon with key 2927324 

Cirsium dissectum
Cirsium dissectum with key 3113784 occurs in 41 km squares.
Cirsium eriophorum
Cirsium eriophorum with key 3112620 occurs in 15 km squares.
Cirsium heterophyllum
Cirsium palustre
Cirsium palustre with key 3113239 occurs in 6119 km squares.
Cirsium tuberosum
Cirsium vulgare
Cirsium vulgare with key 3112801 occurs in 7746 km squares.
Coleonema pulchrum
Cynanchum acutum
Datura wrightii
Dimorphotheca fruticosa
Drosanthemum floribundum
Encelia californica
Eriophyllum confertiflorum
Eupatorium cannabinum
Eupatorium cannabinum with key 5402943 occurs in 6993 km squares.
Euphorbia lamarckii
Euryops chrysanthemoides
Frankenia salina
Hedera helix
Hedera helix with key 8351737 occurs in 7040 km squares.
Heliotropium curassavicum
Helminthotheca echioides
Helminthotheca echioides with key 3141855 occurs in 1445 km squares.
Hirschfeldia incana
Lepidium latifolium
Lepidium latifolium with key 5376692 occurs in 76 km squares.
Lobularia maritima
Lobularia maritima with key 5373358 occ

Geranium dissectum
Geranium dissectum with key 2890040 occurs in 4454 km squares.
Geranium endressii
Geranium endressii with key 2890502 occurs in 150 km squares.
Geranium molle
Geranium molle with key 2891488 occurs in 5627 km squares.
Geranium phaeum
Geranium phaeum with key 2891390 occurs in 299 km squares.
Geranium pratense
Geranium pratense with key 2891304 occurs in 187 km squares.
Geranium pyrenaicum
Geranium pyrenaicum with key 2890122 occurs in 2523 km squares.
Geranium robertianum
Geranium robertianum with key 2890668 occurs in 6893 km squares.
Geranium sanguineum
Geranium sanguineum with key 2890920 occurs in 38 km squares.
Geranium sylvaticum
Geranium sylvaticum with key 2891147 occurs in 32 km squares.
Glechoma hederacea
Glechoma hederacea with key 2926557 occurs in 8719 km squares.
Helleborus niger
Hyacinthoides hispanica
Hyacinthoides hispanica with key 5304257 occurs in 449 km squares.
Hyacinthoides non-scripta
Hyacinthoides non-scripta with key 5304283 occurs in 1952 k

In [699]:
# Convert to a Pandas dataframe
taxaFoundInCountry = pd.DataFrame(taxaFoundInCountry)    

In [700]:
print("The number of species left in the network is {0}".format(len(taxaFoundInCountry)))

The number of species left in the network is 677


In [701]:
taxaFoundInCountry

,key,species,count
0,1283774,Stenarella domator,11
1,5041466,Sceliphron caementarium,1
2,2480830,Bubulcus ibis,1383
3,5071352,Helophilus pendulus,16
4,1539177,Sphaerophoria scripta,15
...,...,...,...
672,3172059,Veronica spicata,20
673,3082234,Cornus sanguinea,3298
674,5389017,Solidago gigantea,2828
675,1423011,Erythromma viridulum,42


## Drawing a network of the interactions

Now that I have a list of all the species in the country I can use this as my nodes list for the network diagram.

In [702]:
#networkx seems to be a leading network tool in Python
import networkx as nx
import matplotlib.pyplot as plt

try:
    import pygraphviz
    from networkx.drawing.nx_agraph import write_dot
    print("using package pygraphviz")
except ImportError:
    try:
        import pydot
        from networkx.drawing.nx_pydot import write_dot
        print("using package pydot")
    except ImportError:
        print()
        print("Both pygraphviz and pydot were not found ")
        print("see  https://networkx.github.io/documentation/latest/reference/drawing.html")
        print()

using package pydot


In [703]:
# Create graphic object
G = nx.DiGraph()

In [704]:
# Match colours to interactions to distinguish them on the graph
colorInteractions = {'interaction':['pollinates', 'mutualistOf', 'eats', 'visitsFlowersOf', 'hasHost', 'parasiteOf', 'pathogenOf'],
        'colour':['r', 'g', 'b', 'y', 'm', 'w', 'c']}  

colorInteractionsDf = pd.DataFrame(colorInteractions)

#len(list(G.nodes))

## A quick look at the interaction data to see if it is what is expected

In [705]:
#cleanInteractDataTaxon.loc[(cleanInteractDataTaxon["sourceTaxonName"] == 'Apis mellifera') & (cleanInteractDataTaxon["targetTaxonName"] == 'Procyon lotor')]
cleanInteractDataTaxon.loc[(cleanInteractDataTaxon["sourceTaxonSpeciesName"] == 'Bombus hortorum')]

,sourceTaxonName,sourceTaxonSpeciesName,sourceTaxonRank,sourceTaxonFamilyName,interactionTypeName,targetTaxonName,targetTaxonSpeciesName,targetTaxonRank,sourceCitation,sourceDOI,sourceTaxonKingdomName,targetTaxonKingdomName
18621,Bombus hortorum,Bombus hortorum,species,Apidae,visitsFlowersOf,Buddleja davidii,Buddleja davidii,species,"Redhead, J.W.; Coombes, C.F.; Dean, H.J.; Dyer...",None,Animalia,Plantae
22028,Bombus hortorum,Bombus hortorum,species,Apidae,visitsFlowersOf,Cirsium arvense,Cirsium arvense,species,"Redhead, J.W.; Coombes, C.F.; Dean, H.J.; Dyer...",None,Animalia,Plantae


## Figure out which node has the most records. This is so that the graphic can be scaled.

In [706]:
if len(taxaFoundInCountry) > 0:
    maxRecords = taxaFoundInCountry.max()["count"]
    print(maxRecords)

15535


### Add the nodes to the graph

In [707]:
dictOfNodeSizes = {}

for index, row in taxaFoundInCountry.iterrows():
     # access data using column names
    #print('A: ', row['species'], row['count'], row['key'])
    G.add_node(row['species'], gbifkey=row['key'], occupancy = row['count'], kingdom = kingdom[row['species']])
    #create a list of node sizes scaled for the network visulization
    #The square root ensures that the node area is proportional to the occupancy, rather than the radius
    dictOfNodeSizes[row['species']] = math.sqrt(int(row['count']/maxRecords*100))*10

In [708]:
cleanInteractDataTaxon

,sourceTaxonName,sourceTaxonSpeciesName,sourceTaxonRank,sourceTaxonFamilyName,interactionTypeName,targetTaxonName,targetTaxonSpeciesName,targetTaxonRank,sourceCitation,sourceDOI,sourceTaxonKingdomName,targetTaxonKingdomName
0,Heterotheca sessiliflora,Heterotheca sessiliflora,species,Asteraceae,visitsFlowersOf,Sceliphron caementarium,Sceliphron caementarium,species,http://iNaturalist.org is a place where you ca...,None,Plantae,Animalia
1,Melittobia chalybii,Melittobia chalybii,species,Eulophidae,parasiteOf,Sceliphron caementarium,Sceliphron caementarium,species,Ed Baker; Ian J. Kitching; George W. Beccaloni...,None,Animalia,Animalia
2,Melittobia megachilis,Melittobia megachilis,species,Eulophidae,parasiteOf,Sceliphron caementarium,Sceliphron caementarium,species,Ed Baker; Ian J. Kitching; George W. Beccaloni...,None,Animalia,Animalia
5,Dibrachys pelos,Dibrachys pelos,species,Pteromalidae,parasiteOf,Sceliphron caementarium,Sceliphron caementarium,species,Ed Baker; Ian J. Kitching; George W. Beccaloni...,None,Animalia,Animalia
6,Monodontomerus montivagus,Monodontomerus montivagus,species,Torymidae,parasiteOf,Sceliphron caementarium,Sceliphron caementarium,species,Ed Baker; Ian J. Kitching; George W. Beccaloni...,None,Animalia,Animalia
...,...,...,...,...,...,...,...,...,...,...,...,...
54725,Bombus pensylvanicus,Bombus pensylvanicus,species,None,hasHost,Verbesina virginica,Verbesina virginica,species,USGS Biodiversity Information Serving Our Nati...,None,None,Plantae
54726,Dielis dorsata,Dielis dorsata,species,Scoliidae,visitsFlowersOf,Verbesina virginica,Verbesina virginica,species,http://iNaturalist.org is a place where you ca...,None,Animalia,Plantae
54727,Papilio glaucus,Papilio glaucus,species,Papilionidae,visitsFlowersOf,Verbesina virginica,Verbesina virginica,species,http://iNaturalist.org is a place where you ca...,None,Animalia,Plantae
54784,Xylocopa virginica,Xylocopa virginica,species,Apidae,visitsFlowersOf,Verbesina virginica,Verbesina virginica,species,http://iNaturalist.org is a place where you ca...,None,Animalia,Plantae


### Add edges to the graph

In [709]:
# iterate over the interacting species that are in GBIF and in the country with iterrows()
# Find the taxa found in the country that are in the source taxon name of the interation data,
# then add the edge if the target species is in the country too.

taxaFoundInCountry_copy = taxaFoundInCountry.copy()

for index, row in taxaFoundInCountry.iterrows():
    # loop over all the taxa finding if any of them are mentioned in the sourceTaxonName field
    for edge in cleanInteractDataTaxon.iterrows():
        if row['species'] == edge[1]['sourceTaxonSpeciesName']:
            #print('B: ', edge[1]['sourceTaxonName'], edge[1]['targetTaxonName'])
            # Some of the target species will not be in GBIF of in the country, so only add an edge where they are.
            for index2, row2 in taxaFoundInCountry_copy.iterrows():
                #print('E: ', row2['species'], edge[1]['targetTaxonName'])
                if row2['species'] == edge[1]['targetTaxonSpeciesName']: 
                    print('C: ', edge[1]['targetTaxonSpeciesName'], row['species'], edge[1]['interactionTypeName'])
                    G.add_edge(row['species'], edge[1]['targetTaxonSpeciesName'], label = edge[1]['interactionTypeName'])
                    
#len(list(G.nodes))


C:  Sceliphron caementarium Stenarella domator parasitoidOf
C:  Achillea millefolium Sceliphron caementarium visitsFlowersOf
C:  Aegopodium podagraria Sceliphron caementarium visits
C:  Allium cepa Sceliphron caementarium visitsFlowersOf
C:  Anethum graveolens Sceliphron caementarium visitsFlowersOf
C:  Anthophora plumipes Sceliphron caementarium createsHabitatFor
C:  Araneus diadematus Sceliphron caementarium preysOn
C:  Araneus diadematus Sceliphron caementarium preysOn
C:  Argiope bruennichi Sceliphron caementarium preysOn
C:  Auplopus carbonarius Sceliphron caementarium createsHabitatFor
C:  Baccharis halimifolia Sceliphron caementarium visitsFlowersOf
C:  Brassica rapa Sceliphron caementarium eats
C:  Buddleja davidii Sceliphron caementarium visitsFlowersOf
C:  Butomus umbellatus Sceliphron caementarium visitsFlowersOf
C:  Campsis radicans Sceliphron caementarium visitsFlowersOf
C:  Campsis radicans Sceliphron caementarium visits
C:  Cirsium arvense Sceliphron caementarium visitsF

C:  Buddleja davidii Episyrphus balteatus visitsFlowersOf
C:  Butomus umbellatus Episyrphus balteatus visitsFlowersOf
C:  Cirsium arvense Episyrphus balteatus visitsFlowersOf
C:  Cirsium arvense Episyrphus balteatus eats
C:  Cirsium arvense Episyrphus balteatus visitsFlowersOf
C:  Daucus carota Episyrphus balteatus visitsFlowersOf
C:  Daucus carota Episyrphus balteatus eats
C:  Daucus carota Episyrphus balteatus visitsFlowersOf
C:  Daucus carota Episyrphus balteatus pollinates
C:  Daucus carota Episyrphus balteatus visitsFlowersOf
C:  Foeniculum vulgare Episyrphus balteatus visitsFlowersOf
C:  Jacobaea vulgaris Episyrphus balteatus visitsFlowersOf
C:  Lysimachia vulgaris Episyrphus balteatus visitsFlowersOf
C:  Lysimachia vulgaris Episyrphus balteatus pollinates
C:  Achillea millefolium Thymelicus sylvestris visitsFlowersOf
C:  Buddleja davidii Thymelicus sylvestris visitsFlowersOf
C:  Cirsium arvense Thymelicus sylvestris visitsFlowersOf
C:  Cirsium arvense Thymelicus sylvestris visit

C:  Foeniculum vulgare Aglais urticae visitsFlowersOf
C:  Solidago canadensis Aglais urticae visitsFlowersOf
C:  Achillea millefolium Sphecodes hyalinatus visitsFlowersOf
C:  Cirsium arvense Sphecodes hyalinatus visitsFlowersOf
C:  Daucus carota Sphecodes hyalinatus visitsFlowersOf
C:  Achillea millefolium Bombus hypnorum visitsFlowersOf
C:  Buddleja davidii Bombus hypnorum visitsFlowersOf
C:  Cirsium arvense Bombus hypnorum visitsFlowersOf
C:  Achillea millefolium Colletes fodiens visitsFlowersOf
C:  Solidago canadensis Colletes fodiens visitsFlowersOf
C:  Achillea millefolium Bombus pascuorum visitsFlowersOf
C:  Buddleja davidii Bombus pascuorum visitsFlowersOf
C:  Cirsium arvense Bombus pascuorum visitsFlowersOf
C:  Cirsium arvense Bombus pascuorum visitsFlowersOf
C:  Cirsium arvense Bombus pascuorum pollinates
C:  Daucus carota Bombus pascuorum visitsFlowersOf
C:  Achillea millefolium Colletes halophilus visitsFlowersOf
C:  Daucus carota Colletes halophilus visitsFlowersOf
C:  Soli

C:  Daucus carota Rhagonycha fulva eats
C:  Daucus carota Rhagonycha fulva visitsFlowersOf
C:  Daucus carota Rhagonycha fulva preysOn
C:  Daucus carota Rhagonycha fulva visits
C:  Jacobaea vulgaris Rhagonycha fulva visitsFlowersOf
C:  Achillea millefolium Cerceris rybyensis visits
C:  Cirsium arvense Cerceris rybyensis visits
C:  Jacobaea vulgaris Cerceris rybyensis visits
C:  Solidago canadensis Cerceris rybyensis visits
C:  Solidago canadensis Cerceris rybyensis visitsFlowersOf
C:  Achillea millefolium Lestica clypeata visits
C:  Achillea millefolium Oxythyrea funesta visitsFlowersOf
C:  Daucus carota Oxythyrea funesta hasHost
C:  Daucus carota Oxythyrea funesta visitsFlowersOf
C:  Jacobaea vulgaris Oxythyrea funesta eats
C:  Achillea millefolium Aricia agestis eats
C:  Buddleja davidii Aricia agestis visitsFlowersOf
C:  Cirsium arvense Aricia agestis visitsFlowersOf
C:  Torilis arvensis Aricia agestis visitsFlowersOf
C:  Achillea millefolium Stenurella melanura eats
C:  Achillea mil

C:  Cirsium arvense Vanessa atalanta visitsFlowersOf
C:  Cirsium arvense Vanessa atalanta visitsFlowersOf
C:  Cirsium arvense Vanessa atalanta pollinates
C:  Cirsium arvense Vanessa atalanta visitsFlowersOf
C:  Cirsium arvense Vanessa atalanta visits
C:  Conium maculatum Vanessa atalanta eats
C:  Daucus carota Vanessa atalanta visitsFlowersOf
C:  Helianthus annuus Vanessa atalanta visitsFlowersOf
C:  Hylotelephium spectabile Vanessa atalanta visitsFlowersOf
C:  Rudbeckia hirta Vanessa atalanta visitsFlowersOf
C:  Solidago canadensis Vanessa atalanta visitsFlowersOf
C:  Torilis arvensis Vanessa atalanta eats
C:  Torilis arvensis Vanessa atalanta visitsFlowersOf
C:  Achillea millefolium Peritrechus geniculatus eats
C:  Daucus carota Peritrechus geniculatus eats
C:  Achillea millefolium Nysius ericae eats
C:  Achillea millefolium Nysius senecionis eats
C:  Achillea millefolium Anisodactylus binotatus eats
C:  Daucus carota Anisodactylus binotatus eats
C:  Achillea millefolium Plagiognathu

C:  Euonymus japonicus Vespa velutina visitsFlowersOf
C:  Foeniculum vulgare Vespa velutina visitsFlowersOf
C:  Argiope bruennichi Dolomedes fimbriatus preysOn
C:  Argiope bruennichi Dolomedes fimbriatus preysOn
C:  Larinioides cornutus Dolomedes fimbriatus preysOn
C:  Larinioides cornutus Dolomedes fimbriatus preysOn
C:  Neoscona adianta Dolomedes fimbriatus preysOn
C:  Neoscona adianta Dolomedes fimbriatus preysOn
C:  Argiope bruennichi Episyron rufipes preysOn
C:  Argiope bruennichi Argiope bruennichi preysOn
C:  Argiope bruennichi Argiope bruennichi eats
C:  Adelphocoris lineolatus Argiope bruennichi preysOn
C:  Adelphocoris quadripunctatus Argiope bruennichi preysOn
C:  Adelphocoris seticornis Argiope bruennichi preysOn
C:  Aeolothrips intermedius Argiope bruennichi preysOn
C:  Aglais io Argiope bruennichi eats
C:  Agrypnus murinus Argiope bruennichi preysOn
C:  Amara lunicollis Argiope bruennichi preysOn
C:  Amblytylus nasutus Argiope bruennichi preysOn
C:  Amphimallon solstitial

C:  Coreopsis verticillata Megachile rotundata eats
C:  Coreopsis verticillata Megachile rotundata visits
C:  Hylotelephium telephium Megachile rotundata visitsFlowersOf
C:  Lotus pedunculatus Megachile rotundata visitsFlowersOf
C:  Medicago sativa Megachile rotundata eats
C:  Trifolium repens Megachile rotundata eats
C:  Trifolium repens Megachile rotundata visitsFlowersOf
C:  Cheiracanthium mildei Auplopus carbonarius preysOn
C:  Baccharis halimifolia Baccharis halimifolia visitsFlowersOf
C:  Baccharis halimifolia Schizophyllum commune hasHost
C:  Baccharis halimifolia Trametes versicolor hasHost
C:  Conium maculatum Trametes versicolor hasHost
C:  Brassica rapa Plebejus argus visitsFlowersOf
C:  Jacobaea vulgaris Plebejus argus visitsFlowersOf
C:  Brassica rapa Bombus bohemicus visitsFlowersOf
C:  Brassica rapa Bombus muscorum visitsFlowersOf
C:  Cirsium arvense Bombus muscorum visitsFlowersOf
C:  Brassica rapa Bombus lucorum visitsFlowersOf
C:  Buddleja davidii Bombus lucorum visit

C:  Hylotelephium spectabile Aglais io eats
C:  Buddleja davidii Tenthredo scrophulariae visits
C:  Buddleja davidii Cydalima perspectalis visitsFlowersOf
C:  Buddleja davidii Pieris mannii visitsFlowersOf
C:  Buddleja davidii Nymphalis antiopa eats
C:  Buddleja davidii Nymphalis antiopa visitsFlowersOf
C:  Cirsium arvense Nymphalis antiopa visitsFlowersOf
C:  Solidago canadensis Nymphalis antiopa visitsFlowersOf
C:  Buddleja davidii Macroglossum stellatarum eats
C:  Buddleja davidii Macroglossum stellatarum visitsFlowersOf
C:  Buddleja davidii Nuctenea umbratica visits
C:  Buddleja davidii Euplagia quadripunctaria visitsFlowersOf
C:  Daucus carota Euplagia quadripunctaria visitsFlowersOf
C:  Buddleja davidii Aedes vexans visitsFlowersOf
C:  Campsis radicans Vespula germanica eats
C:  Campsis radicans Vespula germanica visits
C:  Campsis radicans Vespula germanica visitsFlowersOf
C:  Foeniculum vulgare Vespula germanica visits
C:  Foeniculum vulgare Vespula germanica eats
C:  Solidago 

C:  Daucus carota Orthops basalis eats
C:  Daucus carota Amara lunicollis eats
C:  Daucus carota Amara aenea eats
C:  Daucus carota Chloephaga picta eats
C:  Daucus carota Chloephaga picta eats
C:  Daucus carota Ectemnius lapidarius visitsFlowersOf
C:  Daucus carota Deraeocoris ruber eats
C:  Euonymus japonicus Mormo maura eats
C:  Foeniculum vulgare Hylaeus pictipes visitsFlowersOf
C:  Foeniculum vulgare Capreolus capreolus eats
C:  Lysimachia vulgaris Capreolus capreolus eats
C:  Foeniculum vulgare Lampides boeticus visitsFlowersOf
C:  Foeniculum vulgare Diplazon laetatorius visits
C:  Foeniculum vulgare Diplazon laetatorius visitsFlowersOf
C:  Foeniculum vulgare Diplazon laetatorius eats
C:  Foeniculum vulgare Ichneumon xanthorius visits
C:  Foeniculum vulgare Vespula rufa visitsFlowersOf
C:  Foeniculum vulgare Aulacorthum solani hasHost
C:  Syritta pipiens Foeniculum vulgare mutualistOf
C:  Helianthus annuus Psittacula krameri eats
C:  Helianthus annuus Psittacula krameri eats
C:  

In [ ]:
# iterate over rows with iterrows()

# Find the taxa found in the country that are in the target taxon name of the interation data,
# then add the edge if the source species is in the country too.

for index, row in taxaFoundInCountry.iterrows():
    for edge in cleanInteractDataTaxon.iterrows():
        if row['species'] == edge[1]['targetTaxonName']:
            #print('D: ', edge[1]['sourceTaxonName'], edge[1]['targetTaxonName'])
            #G.add_node(edge[1]['sourceTaxonName'], gbifkey=row['key'])
            #dictOfNodeSizes[edge[1]['sourceTaxonName']] = int(row['count']/maxRecords*100)
            for index2, row2 in taxaFoundInCountry.iterrows():
                #print('E: ', row2['species'])
                if row2['species'] == edge[1]['sourceTaxonSpeciesName']: 
                    print('F: ', edge[1]['sourceTaxonSpeciesName'],edge[1]['targetTaxonSpeciesName'],edge[1]['interactionTypeName'])
                    G.add_edge(edge[1]['sourceTaxonSpeciesName'], row2['species'], label = edge[1]['interactionTypeName'])

F:  Stenarella domator Sceliphron caementarium parasitoidOf
F:  Achillea millefolium Syritta pipiens mutualistOf
F:  Foeniculum vulgare Syritta pipiens mutualistOf
F:  Argiope bruennichi Maniola jurtina eats
F:  Misumena vatia Maniola jurtina eats
F:  Araneus diadematus Pyronia tithonus eats
F:  Misumena vatia Pyronia tithonus eats
F:  Misumena vatia Pieris napi eats
F:  Larinioides cornutus Episyrphus balteatus preysOn
F:  Larinioides cornutus Episyrphus balteatus preysOn
F:  Misumena vatia Episyrphus balteatus eats
F:  Misumena vatia Episyrphus balteatus preysOn
F:  Neoscona adianta Episyrphus balteatus preysOn
F:  Neoscona adianta Episyrphus balteatus preysOn


In [ ]:
print("Number of nodes = {0}".format(G.number_of_nodes()))
print("Number of edges = {0}".format(G.number_of_edges()))

In [ ]:
len(dictOfNodeSizes)

## Remove any nodes that have no edges.
This happens because some of the linking nodes have few supporting observations and so have been weeded out


In [ ]:
#for n in G.nodes:
#    if G.degree(n) == 0:
#        G.remove_node(n)
        
G.remove_nodes_from(list(nx.isolates(G)))

In [ ]:
len(G.nodes)

## Remove any selfloop edges

In [ ]:
nx.number_of_selfloops(G)

In [ ]:
#for e in G.selfloop_edges(data=False): #seems to have changed with a new version of NetworkX
for e in nx.selfloop_edges(G):
    G.remove_edge(e[0],e[1])

In [ ]:
nx.number_of_selfloops(G)

## Run through the list of nodes again and construct a list of the node sizes in the correct order

In [ ]:
listOfNodeSizes = []

for node in G.nodes:
    listOfNodeSizes.append(dictOfNodeSizes[node])

## A network of all the interacting taxa

In [ ]:
plt.figure(figsize=(15,15))
edge_labels = nx.get_edge_attributes(G,'label')

#pos = nx.spring_layout(G, iterations=50, k=50) 
#pos = nx.spring_layout(G)
#pos = nx.random_layout(G)
pos = nx.circular_layout(G)
#pos = nx.spectral_layout(G)
#pos = nx.shell_layout(G, scale=1)

nodeColors = nx.get_node_attributes(G,'color')

nx.draw_networkx_edge_labels(G,pos, edge_labels = edge_labels, font_size=10, font_color='blue')

#nx.draw_networkx_nodes(G, pos, node_color=nodeColors.values())

nx.draw_networkx(G, pos, with_labels=True, node_size = listOfNodeSizes, node_color='c', alpha= 1, arrows=True, 
                    linewidths=1, font_color="black", font_size=10, style = 'dashed')

plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
filename = "..\\docs\\"+taxon+country+".html"
with open(filename, "w") as file:
    file.write(" \
<!DOCTYPE html> \
<html> \
<head> \
<script src='cytoscape.min.js'></script> \
<script src='https://unpkg.com/layout-base/layout-base.js'></script> \
<script src='https://unpkg.com/cose-base/cose-base.js'></script> \
<script src='cytoscape-cose-bilkent.js'></script> \
</head> \
<style>#cy {width: 90%; height: 90%; position: absolute; top: 50px; left: 150px;}\
.red {color: red; font-style: bold;} \
.green {color: green; font-style: bold;} \
.blue {color: blue; font-style: bold;} \
.brown {color: brown; font-style: bold;} \
table, th, td {border: 1px solid black;} \
body {font-family: 'times; font-size: 6px;}\
</style> \
<body> \
<h1><em font-style: italic;>"+taxon+"</em> in "+country+"</h1>")

### Write out the details of the species

In [ ]:
with open(filename, "a") as file:
    if len(G.nodes)==0:
        file.write("<p>No interacting species have been found for "+taxon+" in "+country+".</p>") 
    else:
        file.write("<p>Nodes and edges can be moved and if clicked, open a tab with species or interaction details</p>") 
        file.write("<p>The target species is the <SPAN CLASS='red'>red node</SPAN>. <em>Plantae</em> are <SPAN CLASS='green'>green</SPAN>, <em>Animalia</em> are <SPAN CLASS='blue'>blue</SPAN> and everything else is <SPAN CLASS='brown'>brown</SPAN></p>") 
        file.write("<p>Node area is proportional to the 1 km<sup>2</sup> occupancy</p>")
    

In [ ]:
species = G.nodes
#with open(filename, "a") as file:
#    for n in species:
#        file.write("<tr><td><a target='_blank' href=https://www.gbif.org/species/"+str(nx.get_node_attributes(G, 'gbifkey')[n])+">"+n+"</a></td><td>"+str(dictOfNodeSizes[n])+"</td>\n")

In [ ]:
with open(filename, "a") as file: 
    if len(sourceInteractions) != 0:
        file.write("<p>Below is a table of all <a href='https://www.globalbioticinteractions.org/'>GloBI</a> interaction records that <em>"+taxon+"</em> has with other taxa</p>")
        file.write("<table><th>Interaction</th><th>number</th>")
        for n in sourceInteractions:
            file.write("<tr><td><a target='_blank' href=https://www.globalbioticinteractions.org/?interactionType="+str(n[0])+"&sourceTaxon="+taxon.replace(" ", "%20")+">"+str(n[0])+"</a></td><td>"+str(n[1])+"</td>\n")
        file.write("</table>") 
    if len(targetInteractions) != 0:
        file.write("<p>Below is a table of all <a href='https://www.globalbioticinteractions.org/'>GloBI</a> interaction records that other taxa have with <em>"+taxon+"</em></p>")
        file.write("<table><th>Interaction</th><th>number</th>")
        for n in targetInteractions:
            file.write("<tr><td><a target='_blank' href=https://www.globalbioticinteractions.org/?interactionType="+str(n[0])+"&targetTaxon="+taxon.replace(" ", "%20")+">"+str(n[0])+"</a></td><td>"+str(n[1])+"</td>\n") 
        file.write("</table>")
    

In [ ]:
file.close()

In [ ]:
with open(filename, "a") as file:
    file.write(" \
<div id='cy'></div> \
<script> \
var cy = cytoscape({ \
  container: document.getElementById('cy'), \n \
  elements: [ \
")

### Write nodes to file

In [ ]:
file = open(filename, "a")
for n in species:
    if n == taxon:
        file.write("{ data: { id: '"+n+"', href: 'https://www.gbif.org/species/"+str(nx.get_node_attributes(G, 'gbifkey')[n])+"', occnum: "+str(dictOfNodeSizes[n])+" }, selectable: true,},\n")
    else:
        file.write("{ data: { id: '"+n+"', kingdom: '"+kingdom[n]+"', href: 'https://www.gbif.org/species/"+str(nx.get_node_attributes(G, 'gbifkey')[n])+"', occnum: "+str(dictOfNodeSizes[n])+" }, selectable: false,},\n")
file.close()

### Write edges to file

In [ ]:
file = open(filename, "a")
for edge in G.edges:
    file.write("{data: {id: '"+edge[0]+edge[1]+"', source: '"+edge[0]+"', target: '"+edge[1]+"', href: 'https://www.globalbioticinteractions.org/?interactionType="+str(nx.get_edge_attributes(G, 'label')[edge])+"&sourceTaxon="+edge[0]+"&targetTaxon="+edge[1]+"', label: '"+nx.get_edge_attributes(G, 'label')[edge]+"'}},\n")
file.close()
    

In [ ]:
with open(filename, "a") as file:
    file.write("], \
style: [ \n\
        { \n\
            selector: 'node', \n\
            style: { \n\
                shape: 'circle', \n\
                'background-color': 'yellow', \n\
                label: 'data(id)', \n\
                'text-valign': 'center', \n\
                'font-family': 'helvetica', \n\
                'font-style': 'italic', \n\
                'font-size': '8px', \n\
                'border-width': '0.5', \n\
                'border-color': 'black', \n\
                'width': 'mapData(occnum, 0, 400, 3, 200)', \n\
                'height': 'mapData(occnum, 0, 400, 3, 200)' \n\
            } \n\
        },  \n\
        {  \n\
            selector: 'edge',  \n\
            style: {  \n\
                label: 'data(label)', \n\
                'font-family': 'helvetica', \n\
                'font-size': '6px', \n\
                'color': 'blue', \n\
                'curve-style': 'bezier', \n\
                'target-arrow-shape': 'triangle',  \n\
                'width': '1' \n\
                } \n\
         },  \n\
         {  \n\
              selector: '[kingdom = \"Plantae\"]',   \n\
              css: {  \n\
                'background-color': 'lightgreen'  \n\
                 }  \n\
        }, \n\
        {  \n\
              selector: '[kingdom = \"Animalia\"]',   \n\
              css: {  \n\
                'background-color': 'lightblue'  \n\
                 }  \n\
        }, \n\
        {  \n\
              selector: ':selected',   \n\
              css: {  \n\
                'line-color': 'red',  \n\
                'shape': 'square',  \n\
                'background-color': 'red'  \n\
                 }  \n\
        }], \n\
layout:  { \n\
            name: 'cose-bilkent', padding: 1, animate: true, gravity: 30, animationDuration: 3000 \n\
     } \n\
} \n\
); \n\
")

In [ ]:
with open(filename, "a") as file:
    file.write("cy.userZoomingEnabled( true ); \n\
    cy.getElementById('"+taxon+"').select(); \n\
    document.write('<br />Degree of node <em>"+fullScientificName+"</em> = '+cy.getElementById('"+taxon+"').degree()); \n\
    cy.on('tap', 'node', function(){ \n\
        try { // your browser may block popups \n\
              window.open( this.data('href') ); \n\
            } catch(e){ // fall back on url change \n\
        window.location.href = this.data('href'); \n\
    } \n\
}); \n\
    cy.on('tap', 'edge', function(){  \n\
        try { // your browser may block popups  \n\
              window.open( this.data('href') );  \n\
            } catch(e){ // fall back on url change  \n\
        window.location.href = this.data('href'); \n\
    } \n\
}); \n\
</script> \n\
")

In [ ]:
with open(filename, "a") as file:
    file.write("</ul> \
        <br /> \
        <br /> \
        <a href='https://github.com/AgentschapPlantentuinMeise/interactias'><img src='../images/Octocat.jpg' alt='GitHub logo' width='20'>Source code</a> \
        </body> \
        </html>")

In [ ]:
file.close()

In [ ]:
#with open(filename, "a") as file:
#    file.write("<h2>References</h2><ul>\n")

In [ ]:
citations = cleanInteractDataTaxon['sourceCitation'].unique()
#file = open(filename, "a")
for ref in citations:
    print("* ",str(ref))
#    file.write("<li>"+str(ref)+"</li>\n")
#file.close()

### For import into Gephi: A nodes and edges file has been replace by the .dot file format

In [ ]:
write_dot(G, "..\\docs\\"+taxon+".dot")

In [ ]:
## Output to GraphML for a more universal transfer format suitable for import into R (igraph)

In [ ]:
nx.write_graphml(G, "..\\docs\\"+taxon+".graphml")

